In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

In [2]:
pressures = np.load("../data/initial kernels/new_kernel/pressure.npy")
a_array = np.load("../data/initial kernels/new_kernel/pore_sizes.npy")
kernel = np.load("../data/initial kernels/new_kernel/kernel.npy")

pressures2 = np.load("../data/initial kernels/Pressure_Silica.npy")
a_array2 = np.load("../data/initial kernels/Size_Kernel_Silica_Adsorption.npy")
kernel2 = np.load("../data/initial kernels/Kernel_Silica_Adsorption.npy")

In [3]:
from sklearn.isotonic import IsotonicRegression
for i in range(a_array.shape[0]):
    if a_array[i] > 99:
        if i % 1  == 0:

            iso_reg = IsotonicRegression(increasing=True)
            y_isotonic = iso_reg.fit_transform(pressures, kernel[i])
            plt.plot(pressures, kernel[i]/max(kernel[i]), label=f"{a_array[i]}", marker=".")
            plt.plot(pressures2, kernel2[i]/max(kernel2[i]), label=f"{a_array2[i]}", marker="*")
            #plt.plot(pressures, y_isotonic, label=f"{a_array[i]}", marker="*")
    # if a_array[i] > 3:
    #     if i % 20  == 0:
    #         plt.plot(pressures, kernel[i], label=f"{a_array[i]}", marker=".")
plt.grid(True)
plt.legend()
plt.xscale("log")
plt.show()

In [10]:
new_kernel = []
for i in range(a_array.shape[0]):
    new_kernel.append(iso_reg.fit_transform(pressures, kernel[i]))

In [14]:
pressures_cut_tabels = []
thresholds = [0.5, 0.8, 0.9, 0.95, 0.99]

for threshold in thresholds:
    pressures_cut_tabel = []
    for i in range(a_array.shape[0]):
        for j in range(pressures.shape[0]):
            if new_kernel[i][j] > max(new_kernel[i])*threshold:
                pressures_cut_tabel.append(pressures[j])
                break
    pressures_cut_tabels.append(pressures_cut_tabel)

In [15]:
def plot_cut_tabels(pressures_cut_tabels):
    for i in range(len(pressures_cut_tabels)):
        #plt.plot(a_array, pressures_cut_tabels[i], marker=".", label=f"threshold = {thresholds[i]} (p/p0)")
        plt.plot(pressures_cut_tabels[i], a_array, marker=".", label=f"threshold = {thresholds[i]} (p/p0)")
        plt.xscale("log")
        plt.yscale("log")
        plt.grid(True)
        plt.xlabel("pore size (nm)")
        plt.ylabel("cut pressure (p/p0)")
        plt.legend()
        plt.show()

plot_cut_tabels(pressures_cut_tabels)

In [16]:
for k in range(len(pressures_cut_tabels)):
    for i in range(len(pressures_cut_tabel)-2, -1 , -1):
        if pressures_cut_tabels[k][i] > pressures_cut_tabels[k][i+1]:
            pressures_cut_tabels[k][i] = pressures_cut_tabels[k][i+1]

In [17]:
plot_cut_tabels(pressures_cut_tabels)

In [18]:
final_tabel = np.zeros((a_array.shape[0], 2))
for i in range(len(pressures_cut_tabels[1])):
    final_tabel[i] = np.array((pressures_cut_tabels[1][i], a_array[i]))
final_tabel

array([[1.99370980e-06, 7.00000000e-01],
       [1.99370980e-06, 7.12500000e-01],
       [3.15812498e-06, 7.25000000e-01],
       [5.00266304e-06, 7.37500000e-01],
       [5.00266304e-06, 7.50000000e-01],
       [7.92450363e-06, 7.62500000e-01],
       [1.25526524e-05, 7.75000000e-01],
       [1.57990893e-05, 7.87500000e-01],
       [2.50267960e-05, 8.00000000e-01],
       [3.96441499e-05, 8.12500000e-01],
       [6.27987245e-05, 8.25000000e-01],
       [7.90380443e-05, 8.37500000e-01],
       [1.25198836e-04, 8.50000000e-01],
       [1.57578628e-04, 8.62500000e-01],
       [3.14166790e-04, 8.75000000e-01],
       [3.95410163e-04, 8.87500000e-01],
       [3.95410163e-04, 9.00000000e-01],
       [3.95410163e-04, 9.25000000e-01],
       [3.95410163e-04, 9.50000000e-01],
       [3.95410163e-04, 9.75000000e-01],
       [3.95410163e-04, 1.00000000e+00],
       [3.95410163e-04, 1.05000000e+00],
       [6.26358392e-04, 1.10000000e+00],
       [1.24876539e-03, 1.15000000e+00],
       [3.133782

In [19]:
np.save(f"../data/initial kernels/new_kernel/cut_tabel", final_tabel)

In [21]:
np.save(f"../data/initial kernels/new_kernel/new_kernel", np.array(new_kernel))